In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing
import statsmodels.formula.api as smapi
import itertools

# Sale price distribution
First step is to look at the target sale price for the training data set, i.e. the column we're trying to predict. 

In [ ]:
target = pd.read_csv('data/train_target.csv')

In [ ]:
target.describe()

The sale price is in hte hundreds of thousands, so let's divide the price by 1000 to get more manageable numbers.

In [ ]:
target = target / 1000

In [ ]:
sns.distplot(target);
plt.title('SalePrice')

In [ ]:
import scipy as sp
sp.stats.skew(target)

In [ ]:
sp.stats.skewtest(target)

The distribution is skewed (as demonstrated by the large z-score (and small pvalue) of teh skewtest). It is right skewed (the skew is positive). Skewed distribution are not ideal for linear models, which often assume a normal distribution. One way to correct for right-skewness is to take the log [1,2]

- [1] http://fmwww.bc.edu/repec/bocode/t/transint.html 
- [2] https://www.r-statistics.com/2013/05/log-transformations-for-skewed-and-wide-distributions-from-practical-data-science-with-r/
- [3] Alexandru Papiu's notebook https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models/commentsnotebook 

We apply the function $x \rightarrow \log(1 + x)$ because it is always positive for $x \geq 0$

In [ ]:
logtarget = np.log1p(target)
print('skewness of logtarget = ', sp.stats.skew(logtarget)[0])
print('skewness test of logtarget = ', sp.stats.skewtest(logtarget))
sns.distplot(logtarget)
plt.title(r'log(1 + SalePrice)')

# Merge the training and test datasets for data preparation
We're going to explore the training dataset and apply some transformations to it (fixing missing values, transforming columns etc). We'll need to apply the same transformations to the test dataset. To make that easy, let's use a class that maintains the training and test datasets and keeps them in sync (so that when we apply a transformation to the full dataset, it's applied automatically to the training and test datasets).

In [ ]:
class DataSet:
    """Helper class to manipulate the training and test datasets seamlessly. 

    Attributes
    ----------
    df : dataframe
         Full data containing both the training and test datasets.
    train: dataframe
         The training dataset, kept in sync with df.
    test: dataframe
         The test dataset, kept in sync with df.
    """
    def __init__(self, raw_train, raw_test):
        self.raw_train = raw_train
        self.raw_test = raw_test
        self.train = self.raw_train.copy()
        self.test = self.raw_test.copy()
        self.df = self.merge(self.raw_train, self.raw_test)
        
    @staticmethod
    def merge(train, test):
        return pd.concat([train, test], axis=0, ignore_index=True)
    
    def split(self, alldf):
        n = self.train.shape[0]
        train = alldf.iloc[:n, :].set_index(self.raw_train.index)
        test = alldf.iloc[n:, :].set_index(self.raw_test.index)
        return train, test    
    
    @property
    def df(self):
        return self._df
    
    @df.setter
    def df(self, dataframe):
        self._df = dataframe
        # Update the train and test datasets
        self.train, self.test = self.split(self._df)
        
    def copy(self):
        """Return a copy of the dataset."""
        ds = DataSet(self.train, self.test)
        ds.raw_train = self.raw_train
        ds.raw_test = self.raw_test
        return ds
    
    def apply(self, func, inplace=False):
        """Apply a function func: dataframe -> dataframe 
        to the dataset and return the transformed dataset. 
        Leave raw data unchanged. 
        """
        df = func(self.df)
        if inplace:
            self.df = df
            return self
        else:
            ds = self.copy()
            ds.df = df
            return ds
    
    def __getattr__(self, attr):
        """Try to get the attribute from the the class, 
        otherwise try to get it from the underlying dataframe.
        """
        if attr in self.__dict__:
            return self.__dict__[attr]
        else:
            try:
                return self.df.__getattr__(attr)
            except AttributeError:
                print("Unable to find attribute {!r} in self nor in self.df".format(attr))
                raise
                        
                    
raw_train = pd.read_csv('data/train_prepared_light.csv')
raw_test = pd.read_csv('data/test_prepared_light.csv')
ds = DataSet(raw_train, raw_test)

In [ ]:
# TESTS
def fixture():
    return DataSet(raw_train, raw_test)

def test_split_merge():
    """Check the merge and split functions"""
    ds = fixture()
    df1, df2 = ds.split(ds.df)
    assert all(df1 == raw_train)
    assert all(df2 == raw_test)
    assert all(ds.merge(df1, df2) == ds.df)

    
def test_synchronization():
    """Check that if we update df then the train and test sets are updated accordingly"""
    ds = fixture()
    ds.df = 2 * ds.df
    assert all(ds.train == 2 * ds.raw_train)
    assert all(ds.test == 2 * ds.raw_test)

    
def test_copy():
    ds1 = fixture()
    ds2 = ds1.copy()
    assert not (ds1 is ds2)
    assert all(ds1.df == ds2.df)
    assert all(ds1.raw_train == ds2.raw_train)
    assert all(ds1.train == ds2.train)
    assert all(ds1.test == ds2.test)

    
def test_apply():
    ds = fixture()
    ds2 = ds.apply(lambda x: x * 2)
    assert not (ds is ds2)
    assert all(ds.df == ds2.df * 2)

    
def test_apply_inplace():
    ds = fixture()
    ds_init = ds.copy()
    ds2 = ds.apply(lambda x: x * 2, inplace=True)
    assert (ds is ds2)
    assert all(ds2.df == ds_init.df * 2)
    assert all(ds2.raw_train == ds_init.raw_train)

def test_getattr():
    """Get an attribute of the underlying dataframe if possible"""
    ds = fixture()
    assert all(ds.columns == ds.df.columns)
    assert ds.shape == ds.df.shape
    
test_split_merge()
test_synchronization()
test_copy()
test_apply()
test_apply_inplace()
test_getattr()

In [ ]:
ds.df.shape

In [ ]:
ncategories = sum(ds.df.dtypes == object)
ncategories

# Features
The dataset is wide with 78 features.

In [ ]:
ds.columns, len(ds.columns)

We've got 3 data types: int, float and object

In [ ]:
ds.df.dtypes.unique()

Split the data between categorical and numerical features

In [ ]:
is_categorical = (ds.df.dtypes == object)
is_numerical = (~ is_categorical)

## Numerical features

Create a numerical dataset to keep track of the features

In [ ]:
dsnum = ds.apply(lambda df: df.loc[:, is_numerical])

In [ ]:
dsnum.columns, len(dsnum.columns)

We've got 36 numerical features. We can use the `describe` method to get some statistics:

In [ ]:
dsnum.describe()

But that's a lot of numbers to digest. Better get started plotting! To help with plotting, but also to improve linear regression models, we're going to standardize our data. But before that we must deal with the NaN values.
http://sebastianraschka.com/Articles/2014_about_feature_scaling.html

### Deal with NaN values 

In [ ]:
dfnum = dsnum.df.copy()

In [ ]:
cols_with_nulls = dfnum.columns[dfnum.isnull().sum() > 0]
cols_with_nulls

In [ ]:
dfnum.shape

In [ ]:
dfnum[cols_with_nulls].isnull().sum().sort_values(ascending=False)
#.plot(kind='bar')

Based on the description, the null values for the `MasVnrArea` should be 0 (no massonry veneer type)

In [ ]:
# We may want to refine this in the future. Perhaps build a model to predict the missing GarageCars from the other features?
median_list = 'LotFrontage', 'BsmtFullBath','BsmtHalfBath', 'GarageCars', 'GarageArea'
zero_list = 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', 'BsmtUnfSF'


In [ ]:
for feature in median_list:
    dfnum[feature].fillna(dfnum[feature].median(), inplace=True) 

In [ ]:
for feature in zero_list:
    dfnum[feature].fillna(0, inplace=True)

For the GarageYrBlt, replace by the year the house was built. 

In [ ]:
dfnum.GarageYrBlt.fillna(dfnum.YearBuilt[dfnum.GarageYrBlt.isnull()], inplace=True)


In [ ]:
dsnum.df = dfnum

# Check that everything is in order
def has_nulls(df):
    return df.isnull().sum().any()

assert not has_nulls(dfnum)
assert not has_nulls(dsnum.df)
assert not has_nulls(dsnum.train)
assert not has_nulls(dsnum.test)

### Standardize the data 

In [ ]:
def standardize(df):
    _values = sk.preprocessing.StandardScaler().fit_transform(df)
    return pd.DataFrame(data=_values, columns=df.columns)

dsnum_t = dsnum.apply(standardize)


### Plot violinplots for each feature 
The violin plots give us some idea of the distribution of data for each feature. We can look for things like skewness, non-normality, and the presence of outliers. 

In [ ]:
def violinplot(df, ax=None):
    if ax is None:
        ax = plt.gca()
    sns.violinplot(df, ax=ax)
    for xlab in ax.get_xticklabels():
        xlab.set_rotation(30)
        


In [ ]:
def featureplot(df, nrows=1, figsize=(12,8), plotfunc=violinplot):
    """Plot the dataframe features"""
    width, height = figsize
    fig, axes = plt.subplots(nrows, 1, figsize=(width, height * nrows));
    i = 0
    plots_per_figure = df.shape[1] // nrows
    if nrows == 1:
        axes = [axes]
    for j, ax in zip(range(plots_per_figure, df.shape[1] + 1, plots_per_figure), axes):
        plotfunc(df.iloc[:, i:j], ax=ax)
        i = j


In [ ]:
dsnum_t.train.shape

Many of the features are higly skewed with very long tails.

In [ ]:
featureplot(dsnum_t.train.iloc[:, 0:9])

Most of these are right skewed as well. BsmtFullBath has some discrete values (number of bathrooms).

In [ ]:
featureplot(dsnum_t.train.iloc[:, 9:18])

Some features, such as `BsmtFinSF2`, are almost constant (blobs with long tail) as can be seen below

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4, 4))
sns.distplot(dsnum_t.train['BsmtFinSF2'], ax=ax)
ax.set_title('Distribution of BsmtFinSF2')


### Drop nearly constant features

In [ ]:
def test_nearly_constant(series):
    counts = series.value_counts()
    max_val_count = max(counts)
    other_val_count = counts.drop(counts.argmax()).sum()
    return other_val_count / max_val_count < 0.25

is_nearly_constant = dsnum_t.train.apply(test_nearly_constant)
is_nearly_constant.value_counts()

In [ ]:
dropme = dsnum_t.columns[is_nearly_constant]
dropme

We're going to drop these nearly constant features. If we want to have more control we can transform them into categorical features (for example, is there a screen porch or not?).

In [ ]:
dsnum_t.columns, dsnum_t.shape

In [ ]:
dsnum_t.df = dsnum_t.df.drop(dropme, axis=1)


In [ ]:
dsnum_t.columns, dsnum_t.shape

### Log transform the other features if they have a high skewness

Using a log transformation for some of the skewed features should help, as illustrated below. We use the raw data (not the standardized one) because we need positive values for the log function (we'll standardize the transformed variables later).

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(8, 4))
sns.distplot(dsnum.train['LotArea'], ax=axes[0])
sns.distplot(np.log1p(dsnum.train['LotArea']), ax=axes[1])


In [ ]:
zfactors = sp.stats.skewtest(dsnum_t.train)[0]
sns.distplot(zfactors)

In [ ]:
is_skewed = np.abs(zfactors) > 10
pd.Series(data=zfactors, index=dsnum_t.df.columns)[is_skewed].sort_values().plot(kind='barh')
plt.title('Z-factor for skewtest')

Check the sign of the skewness for all these

In [ ]:
assert all(np.sign(sp.stats.skew(dfnum_t)[is_skewed]) > 0)

Let's apply a log1p transform to all these and plot the distributions again

In [ ]:
def transform_skewed_colums(dfnum, dropme=dropme, is_skewed=is_skewed):
    """
    dfnum: dataframe to transform
    dropme: columns to drop
    is_skewed: iterable of length dfnum.columns indicating if a column is skewed
    """
    dfnum2 = dfnum.copy()
    for feature, skewed_feature in zip(dfnum.columns, is_skewed):
        if skewed_feature:
            dfnum2[feature] = np.log1p(dfnum[feature])

    dfnum_t2 = standardize(dfnum2).drop(dropme, axis=1)
    return dfnum_t2

# the transformed dataset has fewer columns and we only want those
dsnum_t2 = dsnum.apply(transform_skewed_colums)

In [ ]:
dsnum_t2.df.iloc[:, is_skewed].columns

In [ ]:
zfactors2 = sp.stats.skewtest(dsnum_t2.train)[0]
pd.Series(data=zfactors2, index=dsnum_t2.columns)[is_skewed].sort_values().plot(kind='barh')

Now our originally skewed features look more symmetric. 

In [ ]:
featureplot(dsnum_t2.train.iloc[:, is_skewed], nrows=2, figsize=(10,5))

In [ ]:
featureplot(dsnum_t2.train.iloc[:, ~is_skewed], nrows=2, figsize=(10, 5))

### Feature selection
We're now in a good position to identify the key numerical features. Those should be hightly correlated with the sale price.

In [ ]:
nfeatures = dsnum_t2.columns
target_t = standardize(logtarget)

In [ ]:
corr = pd.DataFrame(dsnum_t2.train.apply(lambda feature: sp.stats.pearsonr(feature, target_t['SalePrice'])),
                   columns=['pearsonr'])
corr['correlation'] = corr['pearsonr'].apply(lambda x: x[0])
corr['pvalue'] = corr['pearsonr'].apply(lambda x: x[1])
corr.drop('pearsonr', axis=1, inplace=True)

In [ ]:
corr.sort_values('pvalue', ascending=False)['correlation'].plot(kind='barh')

In [ ]:
corr.sort_values('pvalue').head()

In [ ]:
corr.sort_values('pvalue').tail()

Let's keep only the features that have a high enough correlation with the price (correlation less than 0.2)

In [ ]:
#corr.filter?
min_correlation = 0.2
key_features = corr[np.abs(corr['correlation'] > min_correlation)].sort_values(by='correlation', ascending=False).index.values
key_features, key_features.size

### Basic linear regression model
We're left with 22 features. The first 4 should all be highly correlated with the price.


In [ ]:
data = dsnum_t2.train.copy()
data['SalePrice'] = target_t

In [ ]:

fig, axes = plt.subplots(2,2,figsize=(10,10))
for feature, ax in zip(key_features[:4], itertools.chain.from_iterable(axes)):
    ax.plot(data[feature], data['SalePrice'], 'o')
    ax.set(xlabel=feature, ylabel='SalePrice')
    


Let's build a simple linear regression model based on these 4 features.

In [ ]:
regression1 = smapi.ols("SalePrice ~ OverallQual + GrLivArea + GarageCars + GarageArea", data=data).fit()
regression1.summary()

R-squared equals 0.79 so it's pretty good for a first try. Let's see what happens if we include all our numerical features.

In [ ]:
data.columns

Statsmodels gets confused with columns that start with a digit, so let's rename that column first

In [ ]:
data['1stFlrSF'].name = 'FlrSF'

In [ ]:
data.rename_axis({'1stFlrSF': 'FirstFlrSF', '2ndFlrSF': 'SndFlrSF'}, axis=1, inplace=True)

In [ ]:
desc = 'SalePrice ~ ' + ' + '.join(data.drop('SalePrice', axis=1))
desc

As can be seen below, using more numerical values improves R-squared to 0.88 which is pretty good, though there's of course a risk of overfitting.

In [ ]:
regression2 = smapi.ols(desc, data=data).fit()
regression2.summary()

## Cross validation

In [ ]:
import sklearn.model_selection

In [ ]:
from sklearn.base import BaseEstimator

In [ ]:

def get_data(X, y):
    df = X.copy()
    df['SalePrice'] = y
    return df

def ols1(X, y):
    data = get_data(X, y)
    return smapi.ols("SalePrice ~ OverallQual + GrLivArea + GarageCars + GarageArea", data=data)

def ols2(X, y):
    data = get_data(X, y)
    return smapi.ols(desc, data=data)

## Test the model 
### Use `sklearn.model_selection.train_test_split` to run some experiments and validate the models

In [ ]:

def rmse(prediction, exact):
    return np.mean((prediction - exact)**2.0)**0.5

def run_experiment(estimator, scoring=rmse):
    Xtrain, Xtest, ytrain, ytest = sk.model_selection.train_test_split(data.drop('SalePrice', axis=1), data['SalePrice'])
    model = estimator(Xtrain, ytrain).fit()
    return scoring(model.predict(Xtest), ytest)

def cross_validate(estimator, cv=5):
    return np.array([run_experiment(estimator) for _ in range(cv)])

for model in [ols1, ols2]:
    errors = cross_validate(model)
    print(errors, errors.mean())

        
        

### Use `sklearn.model_selection_cross_val_score` to validate the models

In [ ]:
class Regressor(BaseEstimator):
    
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, X, y):
        self.model = self.estimator(X, y).fit()
    
    def predict(self, X):
        return self.model.predict(X)
    

for model in [ols1, ols2]:
    mse = np.sqrt(-sk.model_selection.cross_val_score(Regressor(model), data.drop('SalePrice', axis=1), y=data['SalePrice'],  
                                   scoring='neg_mean_squared_error', cv=5))
    print(mse, mse.mean())

## Make a submission

In [ ]:
dsnum_t2_submission = dsnum_t2.apply(lambda df: df.rename_axis({'1stFlrSF': 'FirstFlrSF', '2ndFlrSF': 'SndFlrSF'}, axis=1))
submission_t = regression2.predict(dsnum_t2_submission.test)

### Scale the result

In [ ]:
def inverse_transform_target(target_t):
    scaler = sk.preprocessing.StandardScaler()
    scaler.fit(logtarget)
    log_submission = scaler.inverse_transform(submission_t)
    return np.expm1(log_submission) * 1000


In [ ]:
submission = inverse_transform_target(submission_t)
submission

In [ ]:
def save(filename, submission):
    df = pd.DataFrame(data={
            "Id": np.arange(len(submission)) + 1461,
            "SalePrice": submission
            })
    df.to_csv(filename, index=False)
    
save('ols_key_numerical_features_only.csv', submission)

## Regression interpretation
Statsmodels has special plots to explore the outcome of a regression model
http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/example_regression_plots.html